In [ ]:
!pip install -q \
  torch torchvision \
  diffusers==0.27.2 \
  transformers==4.38.2 \
  huggingface_hub==0.20.3 \
  accelerate==0.27.2 \
  peft==0.9.0 \
  safetensors \
  pillow \
  tqdm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.7/130.7 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 108.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.1/330.1 kB 32.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 kB 20.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 4.0.0 requires huggingface-hub>=0.24.0, but you have huggingface-hub 0.20.3 which is incompatible.
gradio 5.50.0 requires huggingface-hub<2.0,>=0.33.5, but you have huggingface-hub 0.20.3 which is incompatible.
sentence-transformers 5.2.0 requires transformers<6.0.0,>=4.41.0, but you have transformers 4.38.2 which is incompatible.


In [ ]:
import diffusers
import transformers
import huggingface_hub
import peft

print("diffusers:", diffusers.__version__)
print("transformers:", transformers.__version__)
print("huggingface_hub:", huggingface_hub.__version__)
print("peft:", peft.__version__)


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

diffusers: 0.27.2
transformers: 4.38.2
huggingface_hub: 0.20.3
peft: 0.9.0


In [ ]:
import os
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
from tqdm import tqdm

from diffusers import StableDiffusionPipeline
from peft import LoraConfig


In [ ]:
BASE_MODEL = "runwayml/stable-diffusion-v1-5"

DATASET_DIR = "/content/drive/MyDrive/Final_year_project/refined_data/real"
OUTPUT_DIR  = "/content/drive/MyDrive/Final_year_project/lora_glioma"

IMAGE_SIZE = 256
BATCH_SIZE = 1
EPOCHS = 10
LR = 1e-4
RANK = 8


In [ ]:
PROMPT = (
    "grayscale MRI scan of human brain showing Neurofibromatosis Type 1, "
    "multiple neurofibromas, abnormal nerve tissue growth, "
    "clinical radiology style, axial view, realistic medical imaging"
)


In [ ]:
NEGATIVE_PROMPT = (
    "text, watermark, logo, labels, annotations, arrows, "
    "cartoon, anime, artistic style, color image, rgb, "
    "distorted anatomy, low quality, blur, noise"
)


In [ ]:
class MedicalMRIDataset(Dataset):
    def __init__(self, image_dir):
        self.images = [
            os.path.join(image_dir, f)
            for f in os.listdir(image_dir)
            if f.endswith(".png")
        ]

        self.transform = transforms.Compose([
            transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
            transforms.ToTensor(),                 # (1, H, W)
            transforms.Normalize([0.5], [0.5])
        ])

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img = Image.open(self.images[idx]).convert("L")
        img = self.transform(img)                  # (1, H, W)
        img = img.repeat(3, 1, 1)                  # 🔑 (3, H, W)
        return img


In [ ]:
dataset = MedicalMRIDataset(DATASET_DIR)
dataloader = DataLoader(
    dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=2,
    pin_memory=True
)

# 🔍 VERIFY SHAPE
images = next(iter(dataloader))
print("IMAGE SHAPE:", images.shape)
assert images.shape[1] == 3, "Dataset is NOT RGB"


IMAGE SHAPE: torch.Size([1, 3, 256, 256])


In [ ]:
pipe = StableDiffusionPipeline.from_pretrained(
    BASE_MODEL,
    dtype=torch.float16,
    safety_checker=None
).to("cuda")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

Fetching 13 files:   0%|          | 0/13 [00:00<?, ?it/s]

scheduler_config.json:   0%|          | 0.00/308 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/342 [00:00<?, ?B/s]

...

diffusion_pytorch_model.safetensors:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

Keyword arguments {'dtype': torch.float16} are not expected by StableDiffusionPipeline and will be ignored.


Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

You have disabled the safety checker for <class 'diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion.StableDiffusionPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .


In [ ]:
pipe.text_encoder.eval()
pipe.text_encoder = pipe.text_encoder.to(dtype=torch.float16)

pipe.unet.train()
pipe.unet = pipe.unet.to(dtype=torch.float16)

pipe.vae.eval()
pipe.vae = pipe.vae.to(dtype=torch.float32)   # VAE MUST BE FP32
pipe.vae.requires_grad_(False)


AutoencoderKL(
  (encoder): Encoder(
    (conv_in): Conv2d(3, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (down_blocks): ModuleList(
      (0): DownEncoderBlock2D(
        (resnets): ModuleList(
          (0-1): 2 x ResnetBlock2D(
            (norm1): GroupNorm(32, 128, eps=1e-06, affine=True)
            (conv1): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (norm2): GroupNorm(32, 128, eps=1e-06, affine=True)
            (dropout): Dropout(p=0.0, inplace=False)
            (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (nonlinearity): SiLU()
          )
        )
        (downsamplers): ModuleList(
          (0): Downsample2D(
            (conv): Conv2d(128, 128, kernel_size=(3, 3), stride=(2, 2))
          )
        )
      )
      (1): DownEncoderBlock2D(
        (resnets): ModuleList(
          (0): ResnetBlock2D(
            (norm1): GroupNorm(32, 128, eps=1e-06, affine=True)
            (c

In [ ]:
lora_config = LoraConfig(
    r=RANK,
    lora_alpha=RANK,
    target_modules=["to_q", "to_k", "to_v", "to_out.0"],
    lora_dropout=0.1,
    bias="none"
)

pipe.unet.add_adapter(lora_config)


In [ ]:
optimizer = torch.optim.AdamW(
    pipe.unet.parameters(),
    lr=LR
)


In [ ]:
print("🚀 STARTING LoRA TRAINING (NF1)")

for epoch in range(EPOCHS):
    total_loss = 0.0

    for images in tqdm(dataloader):
        # VAE expects FP32 RGB
        images = images.to("cuda", dtype=torch.float32)

        # ---- VAE ENCODE ----
        with torch.no_grad():
            latents = pipe.vae.encode(images).latent_dist.sample()
            latents = latents * 0.18215

        # UNet works in FP16
        latents = latents.to(dtype=torch.float16)

        noise = torch.randn_like(latents)
        timesteps = torch.randint(
            0,
            pipe.scheduler.config.num_train_timesteps,
            (latents.shape[0],),
            device="cuda"
        ).long()

        noisy_latents = pipe.scheduler.add_noise(latents, noise, timesteps)

        # ---- TEXT ENCODER ----
        with torch.no_grad():
            text_ids = pipe.tokenizer(
                PROMPT,
                padding="max_length",
                max_length=pipe.tokenizer.model_max_length,
                return_tensors="pt"
            ).input_ids.to("cuda")

            encoder_hidden_states = pipe.text_encoder(text_ids)[0]

        # ---- UNET ----
        noise_pred = pipe.unet(
            noisy_latents,
            timesteps,
            encoder_hidden_states
        ).sample

        loss = torch.nn.functional.mse_loss(noise_pred, noise)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}/{EPOCHS} | Loss: {total_loss:.4f}")





🚀 STARTING LoRA TRAINING (NF1)


  1%|          | 6/944 [00:01<04:21,  3.59it/s]

In [ ]:
os.makedirs(OUTPUT_DIR, exist_ok=True)

pipe.unet.save_pretrained(OUTPUT_DIR)

print("\n✅ NF1 LoRA TRAINING COMPLETE")
print("📁 Saved at:", OUTPUT_DIR)



✅ NF1 LoRA TRAINING COMPLETE
📁 Saved at: /content/drive/MyDrive/Final_year_project/lora_glioma


In [ ]:
from peft.utils import get_peft_model_state_dict
import safetensors.torch as st


In [ ]:
os.makedirs(OUTPUT_DIR, exist_ok=True)

lora_state_dict = get_peft_model_state_dict(pipe.unet)

st.save_file(
    lora_state_dict,
    os.path.join(OUTPUT_DIR, "adapter_model.safetensors")
)

print("✅ adapter_model.safetensors saved successfully")


✅ adapter_model.safetensors saved successfully


In [ ]:
NEXT – NF1 INFERENCE CODE (LOAD LoRA + NEGATIVE PROMPT)